# Animal-AI Configuration tutorial

This tutorial is an introduction of configurating arenas in the Animal-AI environment. 

**We present the format and content of configuration files and show an example of how to generate your own arenas by provided scripts.**

## Introduction of arena configurations

The Animal-AI environment provides a 3d physical arena with an agent that can move and a number of types of objects. The initial status of arenas is configured by `yaml` configuration files. Such files contain:

- experiment parameters (pass mark, maximum steps, steps at which the light is turned on/off)
- a list of arenas
    - a number of object types
        - a list of objects
        - object specifications (positions, rotations, sizes, colors) which are randomized if not provided

Below is an example configuration file:

In [3]:
with open('example1.yaml') as f:
    print(f.read())

#Example 1 showing basic usage.
!ArenaConfig
arenas:
  0: !Arena
    t: 250
    items:
    - !Item
      name: Wall
      positions: 
      - !Vector3 {x: 21, y: 0, z: 10}
      rotations: [0]
      sizes: 
      - !Vector3 {x: 5, y: 3, z: 1}
      colors: 
      - !RGB {r: 255, g: 0, b: 255}
    - !Item
      name: GoodGoalBounce
      positions: 
      - !Vector3 {x: 18, y: 0, z: 22}
      - !Vector3 {x: 26, y: 0, z: 15}
      rotations: [90, 60]
      sizes: 
      - !Vector3 {x: 1, y: 1, z: 1}
      - !Vector3 {x: 3, y: 3, z: 3}
    - !Item
      name: Agent
      positions: 
      - !Vector3 {x: 20, y: 0, z: 15}
      rotations: [0]



This file contains the configuration of one arena (`!Arena`), with an agent `Agent`, two green moving spheres `GoodGoalBounce` and a wall `Wall`. Note that two spheres are declared at once.


You can now use this to load an environment and play yourself (`load_config_and_play.py` does that for you). Make sure you have followed the [installation guide](https://github.com/beyretb/AnimalAI-Olympics#requirements) and then create an `AnimalAIEnvironment` in play mode:

In [ ]:
from animalai.envs.arena_config import ArenaConfig
from animalai.envs.environment import AnimalAIEnvironment
from mlagents_envs.exception import UnityCommunicationException

try:
    environment = AnimalAIEnvironment(
            file_name='env/AnimalAI',
            base_port=5005,
            arenas_configurations=ArenaConfig('example1.yaml'),
            play=True,
        )
except UnityCommunicationException:
    # you'll end up here if you close the environment window directly
    # always try to close it from script
    environment.close()

Press **C** to change the viewpoint (bird's eye, first person, third person), and move with **W,A,S,D** or the **arrows** on your keyboard. 

Press **R** to reset the arena. Note that every time this arena reset, it has **same setup** (rotations, colors, positions, sizes). It is for reason that we explicitly set all configurable attributes of each object in this arena.

Once you're done, let's close this environment:

In [ ]:
if environment:
    environment.close() # takes a few seconds

## Write arena configurations 

This section presents how to use provided scripts to design arenas.

We use functions in `generate_configs.py` to write new configuration files.
First we need to instantiate an `AAIObj`, which holds attributes of target objects as below.

In [5]:
from generate_configs import AAIObj

# Create a purple wall
wall = AAIObj("Wall", positions=[(21,0,10)],rotations=[0],sizes=[(5,3,1)],colors=[(255,0,255)])

We may configure **position**, **rotation**, **size** and **color** of an object. For those attributes not explicitly set, they will be randomized each time arena resets (by pressing **R** in play mode).

The position is (x, y, z) float values, where y is the altitude. The middle of arena is (20, 0, 20).

The rotation is in 0 ~ 360 float values. It is possible to set to 720 or -720, but it might not function properly. It decides the direction object is facing initially.

The size is a float value. 

The color is (R, G, B) float values in 0~255.

Several different objects of same type can be configured at once as below.

In [6]:
# Create two green moving spheres at once
spheres = AAIObj("GoodGoalBounce", positions=[(18,0,22),(26,0,15)],rotations=[90, 60],sizes=[(1,1,1), (3,3,3)])

Note that we do not need to configure color for `spheres` since "GoodGoalBounce" is always green regardless of `colors` values.

Then we need an agent.

In [7]:
# Create an agent
agent = AAIObj("Agent",positions=[(20,0,15)],rotations=[0])

Again, we cannot change the size and color of an agent. It is always (1,1,1) large and in cyan.

Now we have a purple warll, two moving green spheres and an agent. We are ready to write the objects into a `yaml` file.

We may use `ConfigWriter` class in `generate_configs.py`. It provides some functions to format object attributes.

In [8]:
from generate_configs import ConfigWriter
# Instantiate 
cw = ConfigWriter()
file1 = "config_first.yaml"

# Write file and arena header
cw.writeHeader(file=file1, passMark=2.0, time=250, notes="An example configure file")

with open(file1) as f:
    print(f.read())

#An example configure file
!ArenaConfig
arenas:
  0: !Arena
    pass_mark: 2.0
    t: 250
    items:



`writeHeader` is useful when we have only one arena in configuration file. It writes the file header and first arena header.

`passMark` is a float value. Agent need to reach that score by collecting goal balls to show it has ideal performance in this arena. 

`time` is an integer value representing time steps limit of this arena. The arena is terminated when reaching time limit. By setting time to 0 to make the arena permanent.

Different arena in one configuration file may have _different_ pass mark and time limit.

`notes` is an optional message at the top of the configuration file.

Then we need to write the objects defined above into configuration file.

In [9]:
# Write objects
objs = [wall, spheres, agent] # put all objects in a list
cw.writeItems(file1, objs) # append objects to the file

with open(file1) as f:
    print(f.read())

#An example configure file
!ArenaConfig
arenas:
  0: !Arena
    pass_mark: 2.0
    t: 250
    items:
    - !Item
      name: Wall
      positions: 
      - !Vector3 {x: 21, y: 0, z: 10}
      rotations: [0]
      sizes: 
      - !Vector3 {x: 5, y: 3, z: 1}
      colors: 
      - !RGB {r: 255, g: 0, b: 255}
    - !Item
      name: GoodGoalBounce
      positions: 
      - !Vector3 {x: 18, y: 0, z: 22}
      - !Vector3 {x: 26, y: 0, z: 15}
      rotations: [90, 60]
      sizes: 
      - !Vector3 {x: 1, y: 1, z: 1}
      - !Vector3 {x: 3, y: 3, z: 3}
    - !Item
      name: Agent
      positions: 
      - !Vector3 {x: 20, y: 0, z: 15}
      rotations: [0]



An arena which has same configuration as the one we see in previous section is configured now.

You may load and play it as before.

In [ ]:
from animalai.envs.arena_config import ArenaConfig
from animalai.envs.environment import AnimalAIEnvironment
from mlagents_envs.exception import UnityCommunicationException
import random

try:
    environment = AnimalAIEnvironment(
            file_name='env/AnimalAI',
            base_port=5005 + 
                random.randint(0, 100), # use a random port so (probably) do not need to wait for previous to close if need to relaunch
            arenas_configurations=ArenaConfig(file1),
            play=True,
        )
except UnityCommunicationException:
    # you'll end up here if you close the environment window directly
    # always try to close it from script
    environment.close()

In [ ]:
if environment:
    environment.close() # takes a few seconds

## Generate your own arenas

In last section, we have seen 4 attributes of objects (positions, rotations, sizes, colors) and two attributes of arenas (pass mark, time). We also present how to write a configuration file with one arena.

In this section, we will have a look at some advanced features which might be helpful when configuring your experiments.

First of all, let's see how arenas randomize unspecified attributes.

In [11]:
from generate_configs import *
cw = ConfigWriter()
file2 = "config_random.yaml"
cw.writeHeader(file2, passMark=1, time=0, notes="Example showing how arena randomize attributes.") 

# Create a fixed agent and a randomized green sphere
objs=[AAIObj("Agent", positions=[(20,0,10)], rotations=[0]), AAIObj("GoodGoal", None,None,None)]
cw.writeItems(file2, objs)

with open(file2) as f:
    print(f.read())

#Example showing how arena randomize attributes.
!ArenaConfig
arenas:
  0: !Arena
    pass_mark: 1
    t: 0
    items:
    - !Item
      name: Agent
      positions: 
      - !Vector3 {x: 20, y: 0, z: 10}
      rotations: [0]
    - !Item
      name: GoodGoal



Load the configuration and play by cell below, press **R** to see the difference.

Remember that the color of "GoodGoal" is always green.

In [ ]:
from animalai.envs.arena_config import ArenaConfig
from animalai.envs.environment import AnimalAIEnvironment
from mlagents_envs.exception import UnityCommunicationException
import random

try:
    environment = AnimalAIEnvironment(
            file_name='env/AnimalAI',
            base_port=5005 + 
                random.randint(0, 100), # use a random port so (probably) do not need to wait for previous to close if need to relaunch
            arenas_configurations=ArenaConfig(file2),
            play=True,
        )
except UnityCommunicationException:
    # you'll end up here if you close the environment window directly
    # always try to close it from script
    environment.close()

In [ ]:
if environment:
    environment.close() # takes a few seconds

As you see, the position, rotation and size of green sphere is randomized each time the area resets.

It is also possible to have an object with some attributes fixed and some randomized; or a wall fixed and another wall randomized by adding two `AAIObj`.

Next, we are going to see some other configurations for arenas and how to add new attributes not available now.

In [13]:
from generate_configs import *
cw = ConfigWriter()
file3 = "config_blackouts.yaml"
#cw.writeHeader(file3, passMark=1, time=0, notes="Example showing advanced arena configurations.") 

# separate file header and arena header
cw.writeConfigHeader(file3,notes="Example showing advanced arena configurations.")

with open(file3) as f:
    print(f.read())

#Example showing advanced arena configurations.
!ArenaConfig
arenas:



The helper function `writeHeader` does not support attributes other than pass mark and time. We need to use two functions `writeConfigHeader` and `writeArenaHeader` to declare configration header and arena header separately.

As you see, `writeConfigHeader` does not have any parameters related to the arena initialized.

In [14]:
cw.writeArenaHeader(file3,passMark=1,time=0,blackouts=[30,50],arena=0,additional=None)

with open(file3) as f:
    print(f.read())

#Example showing advanced arena configurations.
!ArenaConfig
arenas:
  0: !Arena
    pass_mark: 1
    t: 0
    blackouts: [30, 50]
    items:



`writeArenaHeader` has 4 parameters and accepts additional parameters.

As dicussed above, pass mark is the score to complete this arena and time is the time limit. 

The `arena` parameter sets the index of current arena. You may put several arenas in one configuration file by calling `writeArenaHeader` multiple times. As a tradition, the first arena is indexed 0, the second is 1, etc.

The `blackouts` parameter accepts a list of integer numbers which sets which frames at which the lights turn off/on. By setting it to [30, 50] means to turn off lights at frame 30 then switch on at frame 50. 
It is also possible to set it to a negative number such as [-10], which turns off/on the lights every 10 frames.

Then we add same objects we have in last section to this arena with blackouts.

In [15]:

# Create a fixed agent and a randomized green sphere
objs=[AAIObj("Agent", positions=[(20,0,10)], rotations=[0]), AAIObj("GoodGoal", None,None,None)]
cw.writeItems(file3, objs)

Now let's load and play this new configuration.

Remember pressing **C** to switch to first-person view to see the blackouts.
You may reset the arena by pressing **R** to watch the blackouts again and find how green sphere is randomized.

In [ ]:
from animalai.envs.arena_config import ArenaConfig
from animalai.envs.environment import AnimalAIEnvironment
from mlagents_envs.exception import UnityCommunicationException
import random

try:
    environment = AnimalAIEnvironment(
            file_name='env/AnimalAI',
            base_port=5005 + 
                random.randint(0, 100), # use a random port so (probably) do not need to wait for previous to close if need to relaunch
            arenas_configurations=ArenaConfig(file3),
            play=True,
        )
except UnityCommunicationException:
    # you'll end up here if you close the environment window directly
    # always try to close it from script
    environment.close()

In [ ]:
if environment:
    environment.close() # takes a few seconds

To better support flexible experiment design, `writeArenaHeader` accepts additional parameters to arenas, not objects.

`additional` takes a list of ("name", values) and append the parameters to the 4 parameters introduced above.

Below is an example of configuring blackouts by `additional`.

In [17]:
from generate_configs import *
cw = ConfigWriter()
file4 = "config_additional.yaml"

# separate file header and arena header
cw.writeConfigHeader(file4,notes="Example showing additional arena parameters.")

# pass blackouts parameters by additional parameter
cw.writeArenaHeader(file4, passMark=1, time=0, 
    blackouts=None, arena=0, additional=[("blackouts", [30,50])])
    
# you may check that the arena header is same as the one in file3
with open(file4) as f:
    print(f.read())
    

# Create a fixed agent and a randomized green sphere
objs=[AAIObj("Agent", positions=[(20,0,10)], rotations=[0]), AAIObj("GoodGoal", None,None,None)]
cw.writeItems(file4, objs)

#Example showing additional arena parameters.
!ArenaConfig
arenas:
  0: !Arena
    pass_mark: 1
    t: 0
    blackouts: [30, 50]
    items:



The class `AAIObj` also accepts additional attributes of objects. However, since object attributes are more complicated, the format is not defined. When `ConfigWriter` meets an object with additional attributes, it calls `writeAdditional` in `AAIObj` and pass the file hanlder. It is adviced to inherit `AAIObj` and define your own `writeAdditional` function.

Below is an example showing how to extend `AAIObj` to write additional attributes.

In [18]:
from generate_configs import *
cw = ConfigWriter()
file5 = "config_attributes.yaml"

# separate file header and arena header
cw.writeConfigHeader(file5,notes="Example showing additional object attributes.")

# pass blackouts parameters by additional parameter
cw.writeArenaHeader(file5, passMark=1, time=0, 
    blackouts=None, arena=0, additional=[("blackouts", [30,50])])

# create a class that extends AAIObj
class AAIObj2(AAIObj):
    def __init__(self, name, positions=None, rotations=None, sizes=None, colors=None, additional=None, warning=True) -> None:
        super().__init__(name=name, positions=positions, rotations=rotations, sizes=sizes, colors=colors, additional=additional, warning=warning)
        
    # define your own function to format the additional attributes
    def writeAdditional(self, file):
        for attribute in self.additional: # additional [("name", [values]), ...]
            name = attribute[0] # "      sizes: \n"
            values = attribute[1] # [(1,1,1), (2,2,2)]
            file.write(name)
            for val in values:
                file.write("      - !Vector3 {{x: {}, y: {}, z: {}}}\n".format(val[0], val[1], val[2]))
            

# Create a fixed agent and two randomized green spheres
# configure size of spheres by additional attribute
name = "      sizes: \n" # name for additional attribute
values = [(1,1,1), (2,2,2)] # values for additional attribute
objs=[AAIObj2("Agent", positions=[(20,0,10)], rotations=[0]), 
    AAIObj2("GoodGoal", None,None,None, additional=[(name, values)])]
cw.writeItems(file5, objs)

with open(file5) as f:
    print(f.read())


#Example showing additional object attributes.
!ArenaConfig
arenas:
  0: !Arena
    pass_mark: 1
    t: 0
    blackouts: [30, 50]
    items:
    - !Item
      name: Agent
      positions: 
      - !Vector3 {x: 20, y: 0, z: 10}
      rotations: [0]
    - !Item
      name: GoodGoal
      sizes: 
      - !Vector3 {x: 1, y: 1, z: 1}
      - !Vector3 {x: 2, y: 2, z: 2}



Load and play the configuration use the cell below. It is expected to have an agent and two green spheres with fixed sizes but randomized locations every time the arena is reset by pressing **R**.

In [ ]:
from animalai.envs.arena_config import ArenaConfig
from animalai.envs.environment import AnimalAIEnvironment
from mlagents_envs.exception import UnityCommunicationException
import random

try:
    environment = AnimalAIEnvironment(
            file_name='env/AnimalAI',
            base_port=5005 + 
                random.randint(0, 100), # use a random port so (probably) do not need to wait for previous to close if need to relaunch
            arenas_configurations=ArenaConfig(file5),
            play=True,
        )
except UnityCommunicationException:
    # you'll end up here if you close the environment window directly
    # always try to close it from script
    environment.close()

In [ ]:
if environment:
    environment.close() # takes a few seconds

Congratulations! You have successfully complete this tutorial. We have seen what is configuration file, how to write it with provided scripts, and how to design your own configuration file to support your experiments.

Now you must be ready to setup your own arenas and cannot wait to start experiments in the Animal-AI environment. You might find two other tutorials helpful. `environment_tutorial` intoduces the Animal-AI environment and the basic functionality of it. `training_tutorial` makes you familiar with the procedure of training an agent using animalai-train. Go and find your treasures!